---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [ ]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [1]:
import pandas as pd
import numpy as np

In [2]:
def separateStateRegionValues(x):
    global curStateName
    
    #print(x[0])
    if (x[0].endswith('[edit]')):
        curStateName = x[0].replace('[edit]','')
        return np.NAN
    else:
        regionName = x[0].split(' (')[0]
        #print({'State': curStateName, 'RegionName': regionName})
        return {'State': curStateName, 'RegionName': regionName}

def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    df = pd.read_csv('course1_downloads/university_towns.txt', sep='\n', header=None)
                                 
    
    return (df.apply(separateStateRegionValues, axis=1)
             .dropna()
             .apply(pd.Series))[['State','RegionName']]
                              

In [86]:
%%timeit -n 1 
get_list_of_university_towns()

142 ms ± 22.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
df = pd.read_excel('gdplev.xls', skiprows=219, usecols='E,G', names=['Quarter','GDP'])

prevGDP = np.NAN
prevQGDPDecline = False
prevQGDPRise = True

def processrow(row):
    global prevGDP
    if (prevGDP != np.NAN):
        if row['GDP'] <= prevGDP:
            row['GDPDecline'] = True
        elif row['GDP'] > prevGDP:
            row['GDPDecline'] = False
    prevGDP = row['GDP']
    return row
df.apply(processrow,axis=1)


,Quarter,GDP,GDPDecline
0,2000q1,12359.1,True
1,2000q2,12592.5,False
2,2000q3,12607.7,False
3,2000q4,12679.3,False
4,2001q1,12643.3,True
5,2001q2,12710.3,False
6,2001q3,12670.1,True
7,2001q4,12705.3,False
8,2002q1,12822.3,False
9,2002q2,12893.0,False


In [70]:
df.shape[0]

66

In [5]:
def get_recession():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls', skiprows=219, usecols='E,G', names=['Quarter','GDP'])
    
    recession = False
    dfr=[]

    for index, row in df.iterrows():
        if (index == 0 or index == (df.shape[0]-1)):
            continue
        if (recession == False and(df.iloc[index-1]['GDP'] > df.iloc[index]['GDP'] > df.iloc[index+1]['GDP'])):
            recession = True
            rstart = df.iloc[index]
            #rstartIndex = index
            #print("rstart" + rstart)
        elif (recession == True and (df.iloc[index-1]['GDP'] < df.iloc[index]['GDP'] < df.iloc[index+1]['GDP'])):
            recession = False
            rend = df.iloc[index]
            #rendindex = index
            dfr.append((rstart, rend))
            #print("rend:" + rend)
    return dfr

In [23]:
get_recession()

[(Quarter     2008q3
  GDP        14891.6
  Name: 34, dtype: object, Quarter     2009q3
  GDP        14402.5
  Name: 38, dtype: object)]

In [6]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    return get_recession()[0][0].Quarter
        
get_recession_start()

'2008q3'

In [7]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
    return get_recession()[0][1].Quarter

get_recession_end()

'2009q3'

In [8]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    r = get_recession()[0]
    start, end = r[0].name, r[1].name
    #print(start, end)
    
    rData = df.iloc[start:end+1]
    return rData.loc[rData['GDP'] == np.min(rData['GDP'])].iloc[0]['Quarter']

get_recession_bottom()

'2009q2'

In [112]:
pd.read_csv('City_Zhvi_AllHomes.csv').columns

Index(['RegionID', 'RegionName', 'State', 'Metro', 'CountyName', 'SizeRank', '1996-04', '1996-05', '1996-06', '1996-07',
       ...
       '2015-11', '2015-12', '2016-01', '2016-02', '2016-03', '2016-04', '2016-05', '2016-06', '2016-07', '2016-08'], dtype='object', length=251)

In [135]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    hdf = pd.read_csv('City_Zhvi_AllHomes.csv')
    
    hdf = hdf.apply(transform_to_quarters, axis=1)
    hdf = hdf.set_index(['State','RegionName'])
    return hdf[hdf.columns[-67:]]

def transform_to_quarters(x):
    colLen = len(x.index)
    for i in range(6,colLen,3):
        x[convert_to_quarter(x.index[i])] = np.mean(x[i: (i+3 if i+3 < colLen else colLen)])
    return x
        

    
convert_housing_data_to_quarters()

KeyboardInterrupt: 

In [132]:
hdf = pd.read_csv('City_Zhvi_AllHomes.csv')
#hdf = hdf.set_index(['State','RegionName'])

hds = hdf[['RegionID','RegionName','State','Metro','CountyName','SizeRank']]
hdf = hdf.T
hdf2 = hdf.drop(['RegionID','RegionName','State','Metro','CountyName','SizeRank'])
hdf2
#hdf2 = hdf[6:]
#hdf2.head(25)
#remove text columns
#hdf = hdf[4:]
#hdf = hdf[hdf.index.name[0:4] == '2000']

,0,1,2,3,4,5,6,7,8,9,...,10720,10721,10722,10723,10724,10725,10726,10727,10728,10729
1996-04,NaN,155000,109700,50000,87200,121600,161100,NaN,224500,77500,...,76400,77700,110200,136500,24800,NaN,66900,NaN,115600,129900
1996-05,NaN,154600,109400,49900,87700,120900,160700,NaN,224900,77200,...,75600,77500,110500,136800,24300,NaN,65800,NaN,116400,130200
1996-06,NaN,154400,109300,49600,88200,120400,160400,NaN,225400,76800,...,74100,77700,110900,137000,24500,NaN,65500,NaN,118000,130300
1996-07,NaN,154200,109300,49400,88400,120300,160100,NaN,226100,76600,...,73100,78600,111100,135200,25000,NaN,65100,NaN,119000,129100
1996-08,NaN,154100,109100,49400,88500,120200,159600,NaN,227100,76300,...,72900,79400,111000,133100,25300,NaN,65600,NaN,118400,127900
1996-09,NaN,154300,109000,49300,88900,120300,159100,NaN,228100,76000,...,73900,79700,111000,132700,25600,NaN,65900,NaN,116900,128000
1996-10,NaN,154300,109000,49300,89400,120500,158700,NaN,229300,75900,...,75600,79100,110900,133600,25500,NaN,65400,NaN,115300,128800
1996-11,NaN,154200,109600,49400,89700,120900,158500,NaN,230900,76000,...,77000,78300,111000,134200,25300,NaN,64500,NaN,114400,129000
1996-12,NaN,154800,110200,49700,90100,121400,158700,NaN,233400,76200,...,78100,78400,111300,136600,24900,NaN,64600,NaN,113100,128900
1997-01,NaN,155900,110800,49600,90700,122100,159800,NaN,236500,76500,...,79300,79500,111700,140200,25100,NaN,65600,NaN,112100,128100


In [133]:
hds

,RegionID,RegionName,State,Metro,CountyName,SizeRank
0,6181,New York,NY,New York,Queens,1
1,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,2
2,17426,Chicago,IL,Chicago,Cook,3
3,13271,Philadelphia,PA,Philadelphia,Philadelphia,4
4,40326,Phoenix,AZ,Phoenix,Maricopa,5
5,18959,Las Vegas,NV,Las Vegas,Clark,6
6,54296,San Diego,CA,San Diego,San Diego,7
7,38128,Dallas,TX,Dallas-Fort Worth,Dallas,8
8,33839,San Jose,CA,San Jose,Santa Clara,9
9,25290,Jacksonville,FL,Jacksonville,Duval,10


In [134]:
hdf2 = hdf2.apply(pd.to_numeric, downcast='integer')
hdf2.dtypes

0        float64
1          int32
2          int32
3          int32
4          int32
5          int32
6          int32
7        float64
8          int32
9          int32
10         int32
11       float64
12       float64
13         int32
14         int32
15         int32
16         int32
17         int32
18         int32
19         int32
20         int32
21       float64
22         int32
23         int32
24         int32
25         int32
26         int32
27         int32
28         int32
29         int32
          ...   
10700    float64
10701      int32
10702    float64
10703      int32
10704      int32
10705      int32
10706    float64
10707      int32
10708      int32
10709      int32
10710      int32
10711      int32
10712    float64
10713      int32
10714      int32
10715      int32
10716    float64
10717    float64
10718      int32
10719    float64
10720      int32
10721      int32
10722      int32
10723      int32
10724      int32
10725    float64
10726      int32
10727    float

In [118]:
hdf2['year'] = hdf2.index.str.slice(0,4)
hdf2['q'] = hdf2.index.str.slice(5,7)

qmap = {'01': 'q1', '02': 'q1', '03': 'q1', '04': 'q2', '05': 'q2', '06': 'q2', '07': 'q3', '08': 'q3', '09': 'q3', '10': 'q4', '11': 'q4', '12': 'q4'}
hdf2['qq'] = hdf2.q.apply(lambda x: qmap[x])
hdf2 = hdf2[pd.to_numeric(hdf2['year']) >= 2000]
#hdf.columns


,0,1,2,3,4,5,6,7,8,9,...,10723,10724,10725,10726,10727,10728,10729,year,q,qq
2000-01,NaN,204400,136800,52700,111000,131700,219200,85100.0,364100,88000,...,171300,35500,101100.0,77600,113900.0,148000.0,150800,2000,01,q1
2000-02,NaN,207000,138300,53100,111700,132600,222900,84500.0,374000,88800,...,173700,35500,101500.0,79400,114000.0,151200.0,151000,2000,02,q1
2000-03,NaN,209800,140100,53200,112800,133500,226600,83800.0,384700,89000,...,176900,35200,102700.0,80600,115800.0,153800.0,151300,2000,03,q1
2000-04,NaN,212300,141900,53400,113700,134100,230200,83600.0,395700,88900,...,180500,35200,103700.0,80200,117600.0,155600.0,151000,2000,04,q2
2000-05,NaN,214500,143700,53700,114300,134400,234400,83800.0,407100,89600,...,183400,35500,105200.0,81000,119100.0,156000.0,150200,2000,05,q2
2000-06,NaN,216600,145300,53800,115100,134600,238500,84200.0,416900,90600,...,185400,35700,107300.0,83800,121100.0,156100.0,150300,2000,06,q2
2000-07,NaN,219000,146700,53800,115600,134800,242000,84500.0,424700,91200,...,187400,36100,110000.0,88700,123900.0,156900.0,151900,2000,07,q3
2000-08,NaN,221100,147900,54100,115900,135400,245300,84900.0,431700,91700,...,189000,36700,111800.0,92400,126200.0,157700.0,153500,2000,08,q3
2000-09,NaN,222800,149000,54500,116500,136000,249000,85200.0,439200,92200,...,190500,37200,112300.0,94000,128100.0,159700.0,154300,2000,09,q3
2000-10,NaN,224300,150400,54700,117200,136400,252500,86000.0,447600,92600,...,192400,37400,112900.0,96300,129600.0,164100.0,154400,2000,10,q4


In [129]:
hdf2['yearqq'] = hdf2.year + hdf2.qq
hdf2 = hdf2.set_index('yearqq')
hdf3 = hdf2.groupby('yearqq').mean().T
hdf3

yearqq,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.154667e+05,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,587200.0
1,2.070667e+05,2.144667e+05,2.209667e+05,2.261667e+05,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,2.619667e+05,2.727000e+05,...,4.980333e+05,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,584050.0
2,1.384000e+05,1.436333e+05,1.478667e+05,1.521333e+05,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,1.755000e+05,1.775667e+05,...,1.926333e+05,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,212000.0
3,5.300000e+04,5.363333e+04,5.413333e+04,5.470000e+04,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,5.913333e+04,6.073333e+04,...,1.137333e+05,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,128700.0
4,1.118333e+05,1.143667e+05,1.160000e+05,1.174000e+05,1.196000e+05,1.215667e+05,1.227000e+05,1.243000e+05,1.265333e+05,1.283667e+05,...,1.642667e+05,1.653667e+05,1.685000e+05,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05,1.914333e+05,195200.0
5,1.326000e+05,1.343667e+05,1.354000e+05,1.370000e+05,1.395333e+05,1.417333e+05,1.433667e+05,1.461333e+05,1.493333e+05,1.509333e+05,...,1.700667e+05,1.734000e+05,1.754667e+05,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05,1.972000e+05,199950.0
6,2.229000e+05,2.343667e+05,2.454333e+05,2.560333e+05,2.672000e+05,2.762667e+05,2.845000e+05,2.919333e+05,3.012333e+05,3.128667e+05,...,4.802000e+05,4.890333e+05,4.964333e+05,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05,5.362333e+05,539750.0
7,8.446667e+04,8.386667e+04,8.486667e+04,8.783333e+04,8.973333e+04,8.930000e+04,8.906667e+04,9.090000e+04,9.256667e+04,9.380000e+04,...,1.066333e+05,1.089000e+05,1.115333e+05,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05,1.446000e+05,149300.0
8,3.742667e+05,4.065667e+05,4.318667e+05,4.555000e+05,4.706667e+05,4.702000e+05,4.568000e+05,4.455667e+05,4.414333e+05,4.577667e+05,...,6.794000e+05,6.970333e+05,7.149333e+05,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05,8.189333e+05,822200.0
9,8.860000e+04,8.970000e+04,9.170000e+04,9.310000e+04,9.440000e+04,9.560000e+04,9.706667e+04,9.906667e+04,1.012333e+05,1.034333e+05,...,1.207667e+05,1.217333e+05,1.231667e+05,1.241667e+05,1.269000e+05,1.301333e+05,1.320000e+05,1.339667e+05,1.372000e+05,139900.0


In [124]:
hdf3 = hdf2.groupby('yearqq').mean()
hdf3.T

yearqq,200001,200002,200003,200004,200005,200006,200007,200008,200009,200010,...,201511,201512,201601,201602,201603,201604,201605,201606,201607,201608
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,573600.0,576200.0,578400.0,582200.0,588000.0,592200.0,592500.0,590200.0,588000.0,586400.0
1,204400.0,207000.0,209800.0,212300.0,214500.0,216600.0,219000.0,221100.0,222800.0,224300.0,...,558200.0,560800.0,562800.0,565600.0,569700.0,574000.0,577800.0,580600.0,583000.0,585100.0
2,136800.0,138300.0,140100.0,141900.0,143700.0,145300.0,146700.0,147900.0,149000.0,150400.0,...,207800.0,206900.0,206200.0,205800.0,206200.0,207300.0,208200.0,209100.0,211000.0,213000.0
3,52700.0,53100.0,53200.0,53400.0,53700.0,53800.0,53800.0,54100.0,54500.0,54700.0,...,122300.0,121600.0,121800.0,123300.0,125200.0,126400.0,127000.0,127400.0,128300.0,129100.0
4,111000.0,111700.0,112800.0,113700.0,114300.0,115100.0,115600.0,115900.0,116500.0,117200.0,...,183800.0,185300.0,186600.0,188000.0,189100.0,190200.0,191300.0,192800.0,194500.0,195900.0
5,131700.0,132600.0,133500.0,134100.0,134400.0,134600.0,134800.0,135400.0,136000.0,136400.0,...,190600.0,192000.0,193600.0,194800.0,195400.0,196100.0,197300.0,198200.0,199300.0,200600.0
6,219200.0,222900.0,226600.0,230200.0,234400.0,238500.0,242000.0,245300.0,249000.0,252500.0,...,525700.0,526700.0,527800.0,529200.0,531000.0,533900.0,536900.0,537900.0,539000.0,540500.0
7,85100.0,84500.0,83800.0,83600.0,83800.0,84200.0,84500.0,84900.0,85200.0,86000.0,...,134600.0,136600.0,138700.0,140600.0,142200.0,143300.0,144500.0,146000.0,148200.0,150400.0
8,364100.0,374000.0,384700.0,395700.0,407100.0,416900.0,424700.0,431700.0,439200.0,447600.0,...,789700.0,792100.0,795800.0,803100.0,811900.0,817600.0,819100.0,820100.0,821700.0,822700.0
9,88000.0,88800.0,89000.0,88900.0,89600.0,90600.0,91200.0,91700.0,92200.0,92600.0,...,132000.0,132500.0,133100.0,133900.0,134900.0,136000.0,137200.0,138400.0,139500.0,140300.0


In [93]:
hdf.head(10)

,0,1,2,3,4,5,6,7,8,9,...,10720,10721,10722,10723,10724,10725,10726,10727,10728,10729
RegionID,6181,12447,17426,13271,40326,18959,54296,38128,33839,25290,...,57212,171874,182023,188693,227014,398292,398343,398496,398839,399114
RegionName,New York,Los Angeles,Chicago,Philadelphia,Phoenix,Las Vegas,San Diego,Dallas,San Jose,Jacksonville,...,Forest Falls,Bois D Arc,Henrico,Diamond Beach,Gruetli Laager,Town of Wrightstown,Urbana,New Denmark,Angels,Holland
State,NY,CA,IL,PA,AZ,NV,CA,TX,CA,FL,...,CA,MO,VA,NJ,TN,WI,NY,WI,CA,WI
Metro,New York,Los Angeles-Long Beach-Anaheim,Chicago,Philadelphia,Phoenix,Las Vegas,San Diego,Dallas-Fort Worth,San Jose,Jacksonville,...,Riverside,Springfield,Richmond,Ocean City,NaN,Green Bay,Corning,Green Bay,NaN,Sheboygan
CountyName,Queens,Los Angeles,Cook,Philadelphia,Maricopa,Clark,San Diego,Dallas,Santa Clara,Duval,...,San Bernardino,Greene,Henrico,Cape May,Grundy,Brown,Steuben,Brown,Calaveras,Sheboygan
SizeRank,1,2,3,4,5,6,7,8,9,10,...,10721,10722,10723,10724,10725,10726,10727,10728,10729,10730
1996-04,NaN,155000,109700,50000,87200,121600,161100,NaN,224500,77500,...,76400,77700,110200,136500,24800,NaN,66900,NaN,115600,129900
1996-05,NaN,154600,109400,49900,87700,120900,160700,NaN,224900,77200,...,75600,77500,110500,136800,24300,NaN,65800,NaN,116400,130200
1996-06,NaN,154400,109300,49600,88200,120400,160400,NaN,225400,76800,...,74100,77700,110900,137000,24500,NaN,65500,NaN,118000,130300
1996-07,NaN,154200,109300,49400,88400,120300,160100,NaN,226100,76600,...,73100,78600,111100,135200,25000,NaN,65100,NaN,119000,129100


In [85]:
hdf2[pd.to_numeric(hdf2['year']) >= 2000]

,0,1,2,3,4,5,6,7,8,9,...,10723,10724,10725,10726,10727,10728,10729,year,q,qq
2000-01,NaN,204400,136800,52700,111000,131700,219200,85100,364100,88000,...,171300,35500,101100,77600,113900,148000,150800,2000,01,q1
2000-02,NaN,207000,138300,53100,111700,132600,222900,84500,374000,88800,...,173700,35500,101500,79400,114000,151200,151000,2000,02,q1
2000-03,NaN,209800,140100,53200,112800,133500,226600,83800,384700,89000,...,176900,35200,102700,80600,115800,153800,151300,2000,03,q1
2000-04,NaN,212300,141900,53400,113700,134100,230200,83600,395700,88900,...,180500,35200,103700,80200,117600,155600,151000,2000,04,q2
2000-05,NaN,214500,143700,53700,114300,134400,234400,83800,407100,89600,...,183400,35500,105200,81000,119100,156000,150200,2000,05,q2
2000-06,NaN,216600,145300,53800,115100,134600,238500,84200,416900,90600,...,185400,35700,107300,83800,121100,156100,150300,2000,06,q2
2000-07,NaN,219000,146700,53800,115600,134800,242000,84500,424700,91200,...,187400,36100,110000,88700,123900,156900,151900,2000,07,q3
2000-08,NaN,221100,147900,54100,115900,135400,245300,84900,431700,91700,...,189000,36700,111800,92400,126200,157700,153500,2000,08,q3
2000-09,NaN,222800,149000,54500,116500,136000,249000,85200,439200,92200,...,190500,37200,112300,94000,128100,159700,154300,2000,09,q3
2000-10,NaN,224300,150400,54700,117200,136400,252500,86000,447600,92600,...,192400,37400,112900,96300,129600,164100,154400,2000,10,q4


In [20]:
def convert_to_quarter(x):
    if (x[5:7] in ['01','02','03']):
        return x[0:4] + 'q1'
    elif (x[5:7] in ['04','05','06']):
        return x[0:4] + 'q2'
    elif (x[5:7] in ['07','08','09']):
        return x[0:4] + 'q3'
    elif (x[5:7] in ['10','11','12']):
        return x[0:4] + 'q4'
    else:
        return x

In [23]:
hdf25['yearq'] = hdf25['index'].apply(convert_to_quarter)
hdf25

State,index,NY,CA,IL,PA,AZ,NV,CA,TX,CA,...,MO,VA,NJ,TN,WI,NY,WI,CA,WI,yearq
RegionName,,New York,Los Angeles,Chicago,Philadelphia,Phoenix,Las Vegas,San Diego,Dallas,San Jose,...,Bois D Arc,Henrico,Diamond Beach,Gruetli Laager,Town of Wrightstown,Urbana,New Denmark,Angels,Holland,
0,1996-04,NaN,155000,109700,50000,87200,121600,161100,NaN,224500,...,77700,110200,136500,24800,NaN,66900,NaN,115600,129900,1996q2
1,1996-05,NaN,154600,109400,49900,87700,120900,160700,NaN,224900,...,77500,110500,136800,24300,NaN,65800,NaN,116400,130200,1996q2
2,1996-06,NaN,154400,109300,49600,88200,120400,160400,NaN,225400,...,77700,110900,137000,24500,NaN,65500,NaN,118000,130300,1996q2
3,1996-07,NaN,154200,109300,49400,88400,120300,160100,NaN,226100,...,78600,111100,135200,25000,NaN,65100,NaN,119000,129100,1996q3
4,1996-08,NaN,154100,109100,49400,88500,120200,159600,NaN,227100,...,79400,111000,133100,25300,NaN,65600,NaN,118400,127900,1996q3
5,1996-09,NaN,154300,109000,49300,88900,120300,159100,NaN,228100,...,79700,111000,132700,25600,NaN,65900,NaN,116900,128000,1996q3
6,1996-10,NaN,154300,109000,49300,89400,120500,158700,NaN,229300,...,79100,110900,133600,25500,NaN,65400,NaN,115300,128800,1996q4
7,1996-11,NaN,154200,109600,49400,89700,120900,158500,NaN,230900,...,78300,111000,134200,25300,NaN,64500,NaN,114400,129000,1996q4
8,1996-12,NaN,154800,110200,49700,90100,121400,158700,NaN,233400,...,78400,111300,136600,24900,NaN,64600,NaN,113100,128900,1996q4


In [26]:
hdf25.yearq.unique()

array(['1996q2', '1996q3', '1996q4', '1997q1', '1997q2', '1997q3',
       '1997q4', '1998q1', '1998q2'], dtype=object)

In [24]:
hdf25.groupby(['yearq']).mean()

C:\Users\satis\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2909: PerformanceWarning: indexing past lexsort depth may impact performance.
  if self.run_code(code, result):


State                AK                                                 \
RegionName Anchor Point Anchorage Fairbanks Homer         Juneau Kenai   
yearq                                                                    
1996q2              NaN       NaN       NaN   NaN            NaN   NaN   
1996q3              NaN       NaN       NaN   NaN            NaN   NaN   
1996q4              NaN       NaN       NaN   NaN            NaN   NaN   
1997q1              NaN       NaN       NaN   NaN            NaN   NaN   
1997q2              NaN       NaN       NaN   NaN            NaN   NaN   
1997q3              NaN       NaN       NaN   NaN  181300.000000   NaN   
1997q4              NaN       NaN       NaN   NaN  182766.666667   NaN   
1998q1              NaN       NaN       NaN   NaN  183500.000000   NaN   
1998q2              NaN       NaN       NaN   NaN  182800.000000   NaN   

State                                            ...                WV  \
RegionName Ketchikan Kodiak Lakes North Pole     ...            Vienna   
yearq                                            ...                     
1996q2           NaN    NaN   NaN        NaN     ...      66533.333333   
1996q3           NaN    NaN   NaN        NaN     ...      66400.000000   
1996q4           NaN    NaN   NaN        NaN     ...      66166.666667   
1997q1           NaN    NaN   NaN        NaN     ...      65766.666667   
1997q2           NaN    NaN   NaN        NaN     ...      67066.666667   
1997q3           NaN    NaN   NaN        NaN     ...      69600.000000   
1997q4           NaN    NaN   NaN        NaN     ...      71200.000000   
1998q1           NaN    NaN   NaN        NaN     ...      72300.000000   
1998q2           NaN    NaN   NaN        NaN     ...      72500.000000   

State                                                      WY            \
RegionName    Washington       Waverly  Williamstown Bar Nunn     Burns   
yearq                                                                     
1996q2      74133.333333  59733.333333  74800.000000      NaN       NaN   
1996q3      76566.666667  58600.000000  77666.666667      NaN       NaN   
1996q4      76700.000000  57233.333333  76900.000000      NaN       NaN   
1997q1      76566.666667  56000.000000  75633.333333      NaN       NaN   
1997q2      78000.000000  56033.333333  74733.333333      NaN       NaN   
1997q3      76933.333333  56233.333333  74133.333333      NaN       NaN   
1997q4      76666.666667  58566.666667  74766.666667      NaN       NaN   
1998q1      78400.000000  61266.666667  77266.666667      NaN       NaN   
1998q2      80400.000000  60800.000000  79600.000000      NaN  111800.0   

State                                                 
RegionName   Casper  Cheyenne Evansville Pine Bluffs  
yearq                                                 
1996q2          NaN       NaN        NaN         NaN  
1996q3          NaN       NaN        NaN         NaN  
1996q4          NaN       NaN        NaN         NaN  
1997q1          NaN       NaN        NaN         NaN  
1997q2          NaN       NaN        NaN         NaN  
1997q3          NaN       NaN        NaN         NaN  
1997q4          NaN       NaN        NaN         NaN  
1998q1          NaN       NaN        NaN         NaN  
1998q2      84000.0  115900.0   112600.0    111000.0  

[9 rows x 10730 columns]

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"